Вот еще один вопрос
https://github.com/evfro/polara/blob/master/polara/recommender/models.py#L848
Ты вычислиаешь predition как 
 `scores = (test_matrix.dot(v)).dot(v.T)`
Где `v` до этого 
 `user_factors, sigma, item_factors = svds(svd_matrix, **svd_params)`
 `item_factors = item_factors.T`
 Для general Matrix-Factorisation (я про ALS, funkSVD case где за модель берется любое расложение R = U * V ) 
 для каждого юзера нам надо решить линейное уравнение вида:
 `Ux * v = Ru` , где `Ru` строчка с интеракциями нового customera. ( я прав кстати в этом утверждении или нет ? 
  ибо я сталкиваюсь с тотальним игнором этого биснес кейса (то что ты у себя в PHD называешь onlne-recomendation ) в очень большом количестве алгоритмов что DEEP что и LINEAR,
 но в тоже время это просто стандартний биснес сценарий - есть эмбединги итемов есть свойства пользователя на уровне 
 фич онлайн и векторов интеракций, а не эмбединг полсователя на прямую )
 
 с СВД все проще и мы можем сказать что `Ux`  ортогонален вектору `Ru  * v` , поскольку мы потом используем 
 сцалярное произведение как метрику сходства , то нам этого достаточно  - ето утверждение не понимаю - интуитиця 
 тут как бы понятна  но я не смог доказать это строго. Более того мне кажется надо считать `Ux` не как  `Ru * v`  а как `Ru * ( s * v )`.
 
 Внизу я предлагаю простой пример 3 х 3 где один пользователь имеет для просты одну интеракцию с одним итемом что по интуции должно давать этому пользователю вектор этого итема. 
 
 Можешь указать мне где я не прав ? Более того я провел эксперимент и на моих данных твоя формула 

 `scores = (test_matrix.dot(v)).dot(v.T)`

работает лучше чем 

 `scores = (test_matrix.dot(dot(s,v))).dot(v.T)`

Но если модель по восстановленому юзеру и по юзеру внутри системы с абсолютно иднтичным набором данных дает разные предикшины векторов то с этим что-то не так.


In [129]:
# Vot basovij primer 

import numpy as np
from scipy import linalg

R = np.array([
     [1,2,3],
     [4,5,6],
     [1,0,0]]
)

U, s, v = linalg.svd(R)

print(f"U={U}")
print(f"s={s}")
print(f"v={v}")

testUser = U[-1,:]
print(f"WARM USER s odnoj interakciej = {testUser} NORM={np.dot(testUser,testUser)}")

reconstructedUser = v[:,0]
print(f"COLD USER odnoj interakciej (reconstrovanij)= {reconstructedUser} NORM={np.dot(reconstructedUser,reconstructedUser)}")
print(f"DOT TEST reconstructedUser x testUser = {np.dot(testUser,reconstructedUser)}")

# The same but based on code. 
reconstructedUser = np.dot( R[-1:].reshape((1,3)) , v.T )[0]
print(f"COLD USER s odnoj interakciej (reconstrovanij)= {reconstructedUser} NORM={np.dot(reconstructedUser,reconstructedUser)}")
print(f"DOT TEST reconstructedUser x testUser = {np.dot(testUser,reconstructedUser)}")


# Use Items matrix as s * v 
sigma = np.zeros((m, n))
for i in range(min(m, n)):
    sigma[i, i] = s[i]
svh = np.dot(sigma,Vh)

reconstructedUser = svh[:,0]
print(f"COLD USER s odnoj interakciej (reconstrovanij)= {reconstructedUser} NORM={np.dot(reconstructedUser,reconstructedUser)}")
print(f"DOT TEST reconstructedUser x testUser = {np.dot(testUser,reconstructedUser)}")

# The same but based on code. 
reconstructedUser = np.dot( R[-1:].reshape((1,3)) , svh.T )[0]
print(f"COLD USER s odnoj interakciej (reconstrovanij)= {reconstructedUser} NORM={np.dot(reconstructedUser,reconstructedUser)}")
print(f"DOT TEST reconstructedUser x testUser = {np.dot(testUser,reconstructedUser)}")


U=[[-0.3856283   0.60700632  0.69486268]
 [-0.92153412 -0.21629532 -0.32247668]
 [-0.04544984 -0.7646958   0.64278663]]
s=[9.51777851 1.1565543  0.27253334]
v=[[-0.43258147 -0.56514524 -0.70248426]
 [-0.88441222  0.11459561  0.45241892]
 [ 0.17518078 -0.8169937   0.54939329]]
WARM USER s odnoj interakciej = [-0.04544984 -0.7646958   0.64278663] NORM=1.0
COLD USER odnoj interakciej (reconstrovanij)= [-0.43258147 -0.88441222  0.17518078] NORM=0.9999999999999998
DOT TEST reconstructedUser x testUser = 0.8085709269795148
COLD USER s odnoj interakciej (reconstrovanij)= [-0.43258147 -0.88441222  0.17518078] NORM=0.9999999999999998
DOT TEST reconstructedUser x testUser = 0.8085709269795148
COLD USER s odnoj interakciej (reconstrovanij)= [-4.1172146  -1.02287075  0.0477426 ] NORM=18.000000000000018
DOT TEST reconstructedUser x testUser = 1.0000000000000002
COLD USER s odnoj interakciej (reconstrovanij)= [-4.1172146  -1.02287075  0.0477426 ] NORM=18.000000000000018
DOT TEST reconstructedUser x 